<a href="https://colab.research.google.com/github/euna-jeong20/Intro_Recommender_System/blob/main/%ED%98%91%EC%97%85%ED%95%84%ED%84%B0%EB%A7%81_%EA%B8%B0%EB%B0%98_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_SGD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Base code : https://yamalab.tistory.com/92

In [ ]:
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [ ]:
class MatrixFactorization():
    def __init__(self, R, k, learning_rate, reg_param, epochs, verbose=False):
        
        """
        :param R: rating matrix
        :param k: latent parameter
        :param learning_rate: alpha on weight update
        :param reg_param: beta on weight update
        :param epochs: training epochs
        :param verbose: print status
        """

        self._R = R
        self._num_users, self._num_items = R.shape      #유저 수와 아이템 개수
        self._k = k         #latent space의 크기
        self._learning_rate = learning_rate
        self._reg_param = reg_param     #regularization 하이퍼파라미터
        self._epochs = epochs
        self._verbose = verbose     #학습과정을 보여주냐 아니냐

    def fit(self):
        """
        training Matrix Factorization : Update matrix latent weight and bias

        참고: self._b에 대한 설명
        - global bias: input R에서 평가가 매겨진 rating의 평균값을 global bias로 사용
        - 정규화 기능. 최종 rating에 음수가 들어가는 것 대신 latent feature에 음수가 포함되도록 해줌.

        :return: training_process
        """

        #latent features 초기화
        self._P = np.random.normal(size=(self._num_users, self._k))     #유저에 대한 latent space
        self._Q = np.random.normal(size=(self._num_items, self._k))     #아이템에 대한 latent space

        #편향 초기화
        self._b_P = np.zeros(self._num_users)
        self._b_Q = np.zeros(self._num_items)
        self._b = np.mean(self._R[np.where(self._R !=0)])

        self._training_precess = []
        for epoch in range(self._epochs):
            #rating이 존재하는 index를 기준으로 training
            xi, yi = self._R.nonzero()      #0이 아닌 부분의 좌표x,y값 출력
            for i, j in zip(xi, yi):
                self.gradient_descent(i, j, self._R[i, j])
            cost = self.cost()
            self._training_precess.append((epoch, cost))

            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0):
                print("Iteration: %d ; cost = %.4f" % (epoch + 1, cost))


    def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """

        # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다.
        # 참고: http://codepractice.tistory.com/90
        xi, yi = self._R.nonzero()
        # predicted = self.get_complete_matrix()
        cost = 0
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - self.get_prediction(x, y), 2)
        return np.sqrt(cost/len(xi))


    def gradient(self, error, i, j):
        """
        gradient of latent feature for GD

        :param error: rating - prediction error
        :param i: user index
        :param j: item index
        :return: gradient of latent feature tuple
        """

        dp = (error * self._Q[j, :]) - (self._reg_param * self._P[i, :])
        dq = (error * self._P[i, :]) - (self._reg_param * self._Q[j, :])
        return dp, dq


    def gradient_descent(self, i, j, rating):
        """
        graident descent function

        :param i: user index of matrix
        :param j: item index of matrix
        :param rating: rating of (i,j)
        """

        # 에러 계산
        prediction = self.get_prediction(i, j)      #만든 유저 latent X 만든 아이템 latent + 편향
        error = rating - prediction

        # 편향 업데이트
        self._b_P[i] += self._learning_rate * (error - self._reg_param * self._b_P[i])
        self._b_Q[j] += self._learning_rate * (error - self._reg_param * self._b_Q[j])

        # 유저 latent , 아이템 latent 업데이트
        dp, dq = self.gradient(error, i, j)
        self._P[i, :] += self._learning_rate * dp
        self._Q[j, :] += self._learning_rate * dq


    def get_prediction(self, i, j):
        """
        get predicted rating: user_i, item_j
        :return: prediction of r_ij
        """
        return self._b + self._b_P[i] + self._b_Q[j] + self._P[i, :].dot(self._Q[j, :].T)       #만든 유저 latent X 만든 아이템 latent + 편향


    def get_complete_matrix(self):
        """
        computer complete matrix PXQ + P.bias + Q.bias + global bias

        - PXQ 행렬에 b_P[:, np.newaxis]를 더하는 것은 각 열마다 bias를 더해주는 것
        - b_Q[np.newaxis:, ]를 더하는 것은 각 행마다 bias를 더해주는 것
        - b를 더하는 것은 각 element마다 bias를 더해주는 것

        - newaxis: 차원을 추가해줌. 1차원인 Latent들로 2차원의 R에 행/열 단위 연산을 해주기위해 차원을 추가하는 것.

        :return: complete matrix R^
        """
        return self._b + self._b_P[:, np.newaxis] + self._b_Q[np.newaxis:, ] + self._P.dot(self._Q.T)



# run example
if __name__ == "__main__":
    # rating matrix - User X Item : (7 X 5)
    R = np.array([
        [1, 0, 0, 1, 3],
        [2, 0, 3, 1, 1],
        [1, 2, 0, 5, 0],
        [1, 0, 0, 4, 4],
        [2, 1, 5, 4, 0],
        [5, 1, 5, 4, 0],
        [0, 0, 0, 1, 0],
    ])

In [ ]:
R = np.array([
    [1, 0, 0, 1, 3],
    [2, 0, 3, 1, 1],
    [1, 2, 0, 5, 0],
    [1, 0, 0, 4, 4],
    [2, 1, 5, 4, 0],
    [5, 1, 5, 4, 0],
    [0, 0, 0, 1, 0],
])

In [ ]:
factorizer = MatrixFactorization(R, k=3, learning_rate=0.01, reg_param=0.01, epochs=100, verbose=True)
factorizer.fit()

Iteration: 10 ; cost = 1.1069
Iteration: 20 ; cost = 0.8315
Iteration: 30 ; cost = 0.6356
Iteration: 40 ; cost = 0.4749
Iteration: 50 ; cost = 0.3495
Iteration: 60 ; cost = 0.2602
Iteration: 70 ; cost = 0.1995
Iteration: 80 ; cost = 0.1576
Iteration: 90 ; cost = 0.1274
Iteration: 100 ; cost = 0.1047


In [ ]:
factorizer.get_complete_matrix()

array([[ 1.06504292,  2.05225422,  1.01889099,  1.20010968,  2.82726451],
       [ 1.94069948,  1.789526  ,  3.09861778,  0.84920086,  1.1395617 ],
       [ 1.03265188,  2.04639538,  3.77597492,  4.91398983,  3.28526234],
       [ 0.99409729,  3.40595781,  2.95657879,  3.9682224 ,  4.02834165],
       [ 2.06806617,  1.11889158,  5.01492758,  3.83927965,  1.45072879],
       [ 4.9241935 ,  0.87717816,  4.94696934,  4.1287508 ,  3.02388713],
       [-0.29576677,  0.19097503,  4.13284098,  1.11876574, -1.5697363 ]])